In [1]:
#카프카로 실시간 정보를 전송하는 라이브러리 설치
!pip install kafka-python

                                              0.0/246.5 kB ? eta -:--:--
                                              0.0/246.5 kB ? eta -:--:--
                                              0.0/246.5 kB ? eta -:--:--
     -                                        10.2/246.5 kB ? eta -:--:--
     -                                        10.2/246.5 kB ? eta -:--:--
     ----                                  30.7/246.5 kB 262.6 kB/s eta 0:00:01
     ----                                  30.7/246.5 kB 262.6 kB/s eta 0:00:01
     ------                                41.0/246.5 kB 245.8 kB/s eta 0:00:01
     -------------                         92.2/246.5 kB 403.5 kB/s eta 0:00:01
     -----------------------              163.8/246.5 kB 656.4 kB/s eta 0:00:01
     ------------------------------------ 246.5/246.5 kB 945.1 kB/s eta 0:00:00


In [3]:
!pip install pandas

                                              0.0/10.7 MB ? eta -:--:--
                                              0.0/10.7 MB ? eta -:--:--
                                              0.0/10.7 MB ? eta -:--:--
                                              0.0/10.7 MB ? eta -:--:--
                                             0.0/10.7 MB 187.9 kB/s eta 0:00:57
                                             0.0/10.7 MB 217.9 kB/s eta 0:00:49
                                             0.1/10.7 MB 297.7 kB/s eta 0:00:36
                                             0.1/10.7 MB 467.6 kB/s eta 0:00:23
                                             0.2/10.7 MB 737.3 kB/s eta 0:00:15
     ---                                      1.0/10.7 MB 3.9 MB/s eta 0:00:03
     --------------                           4.0/10.7 MB 12.1 MB/s eta 0:00:01
     ---------------                          4.2/10.7 MB 12.7 MB/s eta 0:00:01
     ---------------                          4.2/10.7 MB 12.7 MB/s eta 0

In [13]:
from kafka import KafkaProducer 
from json import dumps
import json
import pandas as pd
import time
import datetime

In [14]:
#버스의 위치 정보가 저장된 location.csv 읽기
location_df = pd.read_csv("location.csv")

In [15]:
location_df 

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91769,302350,2019-11-05,405328102,7983486,281-2,33.493625,126.534764,제주지방법원(광양방면),20시,272.0,고산동산(광양방면),33.495540,126.532907
91770,302351,2019-11-05,405328102,7983486,281-2,33.495540,126.532907,고산동산(광양방면),20시,447.0,제주시청(광양방면),33.498925,126.530351
91771,302352,2019-11-05,405328102,7983486,281-2,33.498925,126.530351,제주시청(광양방면),20시,418.0,광양사거리,33.500473,126.527103
91772,302353,2019-11-05,405328102,7983486,281-2,33.500473,126.527103,광양사거리,20시,140.0,탐라장애인 종합복지관,33.500228,126.525625


In [16]:
#0번째 버스 위치 정보 조회
location_df.loc[0]

id                     210457
date               2019-10-29
route_id            405136001
vh_id                 7997025
route_nm                360-1
now_latitude        33.457724
now_longitude      126.554014
now_station              제대마을
now_arrive_time           07시
distance                333.0
next_station            제대아파트
next_latitude       33.458783
next_longitude     126.557353
Name: 0, dtype: object

In [17]:
#0번째 버스 위치 정보를 JSON으로 변환
location_df.loc[0].to_json(force_ascii=False)

'{"id":210457,"date":"2019-10-29","route_id":405136001,"vh_id":7997025,"route_nm":"360-1","now_latitude":33.457724,"now_longitude":126.554014,"now_station":"제대마을","now_arrive_time":"07시","distance":333.0,"next_station":"제대아파트","next_latitude":33.458783,"next_longitude":126.557353}'

In [18]:
producer=KafkaProducer(
            #메시지 받은 사람이 메시지를 잘 받았는지 체크하는 옵션 
            #(0은 그냥 보내기만 한다. 확인x)
            acks=0, 
            #메시지 전달할 때 압축
            compression_type='gzip', 
            #현제 위치를 저장할 카프카 아이피
            bootstrap_servers=['192.168.56.101:9092'], 
            #직렬화 : 데이터 전송을 위해 byte단위로 바꿔주는 작업 : 
            #dumps 함수이용. dump : json 값을 메모리에 올려준다.
            #encode를 통해서 올린다.x가 있으면, x를 dumps로 바꾸고
            #utf-8 로 설정한다
            value_serializer=lambda x: dumps(x).encode('utf-8') 
          )

In [ ]:
#카프카로 전송 할 데이터의 수
count = len(location_df)
count

In [ ]:
#인덱스의 시작
index = 0
#무한 반복됨
while True:
    #index가 count 이상일때
    if index >= count :
        #index를 0으로 초기화
        index = 0
    #현재 날짜와 시간을 조회
    now = datetime.datetime.now()
    print("now = ", now)
    #index번째 row를 location_row에 저장
    location_row = location_df.loc[index]
    #location_row를 JSON 타입 문자열로 변환해서 data에 저장
    data = location_row.to_json(force_ascii=False)
    #문자열 data를 JSON 객체로 변환
    data = json.loads(data)
    #현재 시간 추가
    data["hour"] = now.hour
    #요일 추가
    data["weekday"] = now.weekday()
    #master 노드의 카프카의 car_location_topic 으로 현재 위치 정보 data를 전송 할 준비를 함
    producer.send('car_location_topic',value=data)
    #현재 위치 정보 data를 master노드 kafaka로 전송
    producer.flush()
    print("data =", data)
    #30초 대기
    time.sleep(30)
    #인덱스 1증가
    index = index + 1
    print("=" * 100)

now =  2023-06-29 13:47:56.319458
data = {'id': 210457, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.457724, 'now_longitude': 126.554014, 'now_station': '제대마을', 'now_arrive_time': '07시', 'distance': 333.0, 'next_station': '제대아파트', 'next_latitude': 33.458783, 'next_longitude': 126.557353, 'hour': 13, 'weekday': 3}
now =  2023-06-29 13:48:26.337329
data = {'id': 210458, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.458783, 'now_longitude': 126.557353, 'now_station': '제대아파트', 'now_arrive_time': '07시', 'distance': 415.0, 'next_station': '제주대학교', 'next_latitude': 33.459893, 'next_longitude': 126.561624, 'hour': 13, 'weekday': 3}
now =  2023-06-29 13:48:56.340211
data = {'id': 210459, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.478867, 'now_longitude': 126.483833, 'now_station': '한라중학교/부영아파트', 'now_arrive_time': '08시', '

now =  2023-06-29 13:56:56.386552
data = {'id': 210475, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.499683, 'now_longitude': 126.521267, 'now_station': '남서광마을입구', 'now_arrive_time': '09시', 'distance': 232.0, 'next_station': '남서광마을', 'next_latitude': 33.499789, 'next_longitude': 126.523759, 'hour': 13, 'weekday': 3}
now =  2023-06-29 13:57:26.390430
data = {'id': 210476, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.499789, 'now_longitude': 126.523759, 'now_station': '남서광마을', 'now_arrive_time': '09시', 'distance': 193.0, 'next_station': '탐라장애인 종합복지관', 'next_latitude': 33.500017, 'next_longitude': 126.525833, 'hour': 13, 'weekday': 3}
now =  2023-06-29 13:57:56.394240
data = {'id': 210477, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.500017, 'now_longitude': 126.525833, 'now_station': '탐라장애인 종합복지관', 'now_arrive_time':

now =  2023-06-29 14:05:56.439524
data = {'id': 210493, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.485662, 'now_longitude': 126.494923, 'now_station': '도호동', 'now_arrive_time': '11시', 'distance': 374.0, 'next_station': '은남동', 'next_latitude': 33.485822, 'next_longitude': 126.490897, 'hour': 14, 'weekday': 3}
now =  2023-06-29 14:06:26.443196
data = {'id': 210494, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.478867, 'now_longitude': 126.483833, 'now_station': '한라중학교/부영아파트', 'now_arrive_time': '11시', 'distance': 417.0, 'next_station': '대림2차아파트', 'next_latitude': 33.47885, 'next_longitude': 126.48835, 'hour': 14, 'weekday': 3}
now =  2023-06-29 14:06:56.446705
data = {'id': 210495, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.47885, 'now_longitude': 126.48835, 'now_station': '대림2차아파트', 'now_arrive_time': '11시', 'dis

now =  2023-06-29 14:14:56.494774
data = {'id': 210511, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.49993, 'now_longitude': 126.514863, 'now_station': '제주버스터미널', 'now_arrive_time': '12시', 'distance': 262.0, 'next_station': '한국병원', 'next_latitude': 33.499793, 'next_longitude': 126.517687, 'hour': 14, 'weekday': 3}
now =  2023-06-29 14:15:26.496339
data = {'id': 210512, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.499793, 'now_longitude': 126.517687, 'now_station': '한국병원', 'now_arrive_time': '12시', 'distance': 333.0, 'next_station': '남서광마을입구', 'next_latitude': 33.499683, 'next_longitude': 126.521267, 'hour': 14, 'weekday': 3}
now =  2023-06-29 14:15:56.499465
data = {'id': 210513, 'date': '2019-10-29', 'route_id': 405136001, 'vh_id': 7997025, 'route_nm': '360-1', 'now_latitude': 33.499683, 'now_longitude': 126.521267, 'now_station': '남서광마을입구', 'now_arrive_time': '12시', 'di